In [2]:
import csv
import os
import pandas as pd
import re
import sys
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [3]:
def process_json(dirname, fname):
    temp_df = pd.read_json(os.path.join(dirname, fname), lines = True).drop(["url"], axis = 1).dropna().replace('\n', ' ', regex=True)

#     # Prepare label in fastText format
#     temp_df["party"] = ['__label__'+s for s in temp_df["party"]]

    return temp_df

# Setup sql
engine = create_engine('sqlite:///wikidb.db')

session = sessionmaker()
session.configure(bind=engine)
s = session()

In [9]:
dirnames = ["data_extract/text/AA", "data_extract/text/AB"]

for dirname in dirnames:
    for fname in os.listdir(dirname):
        temp_df = process_json(dirname, fname)

        temp_df.to_sql("wikidb", con = engine, index = False, if_exists = "append", chunksize = 1000)

    #     temp_df.to_csv(f'{os.path.basename(fname)}.txt', index=False, sep=' ', 
    #                    header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

        del temp_df

    s.commit()

In [11]:
# Test that the db exists

pd.read_sql_query(f'SELECT COUNT(*) FROM wikidb', engine)

,COUNT(*)
0,143411
